### Split Transmission Line Dataset by Voltage Class
This notebook is the first notebook in the `Phase 3: Transmission Modelling` workflow. We split the Homeand Infrastructure Foundation Level Database (HIFLD) Transmission_Lines dataset into 8 geodatabase feature classes based on their 'VOLT_CLASS' attribute. For subsequent analyses, we concern ourselves with the 5 most common voltage classes: under_100, 100_161v, 220-287v, 345v, and 500v. 

In [53]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Set Input and Output Paths

In [54]:
# Set paths to input and output workspaces
mainInputFolder = 'C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\YifanData'
output_workspace = 'C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\TransmissionLines.gdb'

arcpy.env.workspace = mainInputFolder
arcpy.env.overwriteOutput = True

In [55]:
# Create output workspace
if not arcpy.Exists(output_workspace):
    arcpy.CreateFileGDB_management(out_folder_path=os.path.dirname(output_workspace),
                                   out_name=os.path.basename(output_workspace))

In [56]:
# Set path to table
table_path = os.path.join(mainInputFolder, 'Electric_Power_Transmission_Lines_project.shp')


#### Identify Voltage Classes

In [ ]:
# Get table fields
field_names = arcpy.ListFields(table_path)
field_list = []
for field in field_names:
    field_list.append(field.name)
    print(field.name)

In [ ]:
# Preview Attribute Table
with arcpy.da.SearchCursor(table_path, field_list) as cursor:
    for row in cursor:
        print(row)

In [59]:
# Create a set to store unique voltage classes (sets have no duplicate elements)
voltage_classes = set()

In [60]:
# Store unique voltage classes in the set
with arcpy.da.SearchCursor(table_path, field_list[14]) as cursor:
    for row in cursor:
        value = row[0]
        voltage_classes.add(value)


In [ ]:
# View the voltage classes
voltage_classes

# Create corresponding voltage list for naming
voltage_list = [voltage_class.replace('-', '_').replace(' ', '_') for voltage_class in voltage_classes]
voltage_list = [voltage.lower() for voltage in voltage_list]
voltage_list

#### Generate Transmission Line Features by Voltage Class

In [ ]:
for voltage_class, voltage_name in zip(voltage_classes, voltage_list):
    
    # Output feature class name
    output_feature_class = "voltage_{}".format(voltage_name)

    # Use the MakeFeatureLayer to create a temp layer 
    arcpy.MakeFeatureLayer_management(table_path, "temp_layer", "VOLT_CLASS = '{}'".format(voltage_class))

    # Copy the temp layer to the output workspace
    arcpy.CopyFeatures_management("temp_layer", "{}\\{}".format(output_workspace, output_feature_class))

    # Delete the temp layer
    arcpy.Delete_management("temp_layer")

    # Print 
    print("Created Feature Class {} in {}".format(output_feature_class, os.path.basename(output_workspace)))

In [ ]:
os.path.basename(output_workspace)